# Step 1: Get movie data

In [2]:
import requests
import json
from collections import defaultdict
import random
import openai


headers = {
    "accept": "application/json",
    "Authorization": "Bearer "
}

openai.api_key = ""

In [45]:
def get_movies(n):
    """
    return a list of n movie_ids
    """
    #randomize between popular and top rated 
    option = random.randint(1,2)
    
    if option == 1: #top rated
        page = random.randint(1, 447)
        url = "https://api.themoviedb.org/3/movie/top_rated?language=en-US&page="+str(page)
    else: #popular
        page = random.randint(1, 500)
        url = "https://api.themoviedb.org/3/movie/popular?language=en-US&page="+str(page)
        
    response = requests.get(url, headers=headers)
    response_data = json.loads(response.text)
    
    random_movies = random.sample(response_data["results"],n )    
    
    movie_ids = []
    for movie in random_movies:
        movie_ids.append(str(movie['id']))
    
    return movie_ids

In [157]:
def generate_description(movie_id, choice):
    
    #generic description based on genre, overview, reviews, tagline 
    if choice in [1,2]:
        text = ""
        #get reviews
        url = "https://api.themoviedb.org/3/movie/" + movie_id + "/reviews?language=en-US&page=1"
        reviews = requests.get(url, headers=headers)
        review_data = json.loads(reviews.text)
        if review_data['results']: 
            for i in range(len(review_data['results'])):
                text += review_data['results'][i]['content']

        #get overview
        url = "https://api.themoviedb.org/3/movie/"+ movie_id+ "?language=en-US"
        overall = requests.get(url, headers=headers)
        overall_data = json.loads(overall.text)
        for genre in overall_data['genres']:
            text += genre['name']
        text += overall_data['tagline']
        text += overall_data['overview']
        if choice == 1:
            prompt = f"generate a 'blind date with a book' description for this movie:\
                    \n\n{text}\n\n keep the description to maximum 15 words. Do NOT mention the movie's title."

            response = openai.Completion.create(
                engine="text-davinci-003",
                prompt=prompt,
                max_tokens=256
            )["choices"][0]["text"]
        else:
            url = "https://api.themoviedb.org/3/movie/"+ movie_id + "/credits?language=en-US"

            credits_response = requests.get(url, headers = headers)
            credits_data = json.loads(credits_response.text)
            actors = credits_data['cast'][0]['name'] + " and " + credits_data['cast'][1]['name'] 
            prompt = f"generate a very short description for this movie:\
                    \n\n{text}\n\n Do NOT mention the movie's title. Make sure to mention BOTH {actors} with their full names.\
                    Keep the description to maximum 15 words. "

            response = openai.Completion.create(
                engine = "text-davinci-003",
                prompt = prompt,
                max_tokens=256
            )["choices"][0]["text"]
    
    
    #description based on movie recommendation 
    elif choice == 3:
        url = "https://api.themoviedb.org/3/movie/"+ movie_id + "/recommendations?language=en-US&page=1"
        recommend_response = requests.get(url, headers=headers)
        recommend_data = json.loads(recommend_response.text)
        if len(recommend_data['results']) > 0:
            recommend_movie = recommend_data['results'][0]['title']
            year = "(" + recommend_data['results'][0]['release_date'][:4] + ")"
            response = f"If you like {recommend_movie} {year}, you'll like this movie"
        else:
            url = "https://api.themoviedb.org/3/movie/"+ movie_id + "/similar?language=en-US&page=1"
            recommend_response = requests.get(url, headers=headers)
            recommend_data = json.loads(recommend_response.text)
            if len(recommend_data['results']) > 0:
                recommend_movie = recommend_data['results'][0]['title']
                response = f"If you like {recommend_movie}, you'll like this movie"
            
            else:
                response = generate_description(movie_id, choice = 1)
        
        
    #description based on movie keywords 
    elif choice == 4:
        url = "https://api.themoviedb.org/3/movie/"+ movie_id + "/keywords"
        keywords_response = requests.get(url, headers = headers)
        keywords_data = json.loads(keywords_response.text)
        keywords_response = random.sample(keywords_data['keywords'], min(4, len(keywords_data['keywords'])))
        keywords_list = []
        response = ""
        
        for item in keywords_response:
            response += item['name'] + ", "
            
        if len(response) == 0:
            response = generate_description(movie_id, choice=1)
        
    response = response.replace('\n','')

    return response

In [151]:
def get_link(movie_id):
    url = "https://api.themoviedb.org/3/movie/"+ movie_id + "/watch/providers"
    response = requests.get(url, headers=headers)
    response_data = json.loads(response.text)
    if "US" in response_data['results']:
        link = response_data['results']['US']['link']
    else:
        random_country_code = random.choice(list(response_data['results'].keys()))
        link = response_data['results'][random_country_code]['link']
    
    return link

In [158]:
def get_random_descriptions():
    movie_ids = get_movies(4)
    
    choices = [1, 2, 3, 4]
    random_choices = random.sample(choices, len(choices))
    
    blind_descriptions = []
    links = []
    for movie_id, choice in zip(movie_ids, random_choices):
        blind_descriptions.append(generate_description(movie_id, choice))
        links.append(get_link(movie_id))
        
    return blind_descriptions, links

In [160]:
get_random_descriptions()

(['prophecy, apocalypse, sacrifice, biblical plagues, ',
  "If you like Mama (2013), you'll like this movie",
  'Love blooms on a college campus.',
  'Parallel universes unravel mystery for Mark Hapka and Jessica Rothe.'],
 ['https://www.themoviedb.org/movie/11082-the-seventh-sign/watch?locale=US',
  'https://www.themoviedb.org/movie/133805-carrie/watch?locale=US',
  'https://www.themoviedb.org/movie/289727-the-rewrite/watch?locale=US',
  'https://www.themoviedb.org/movie/328595-parallels/watch?locale=US'])

In [154]:
for i in range(5):
    print(get_random_descriptions())

(['Asa Butterfield and Jack Scanlon uncover a heartbreaking history in a divided land.', "If you like Heb Medelij Jet!, you'll like this movie", 'Change fate with fantasy and adventure.', 'stargazing, parent-in-law child-in-law relationship, 1950s, family, '], ['https://www.themoviedb.org/movie/14574-the-boy-in-the-striped-pyjamas/watch?locale=US', 'https://www.themoviedb.org/movie/297-meet-joe-black/watch?locale=US', 'https://www.themoviedb.org/movie/62177-brave/watch?locale=US', 'https://www.themoviedb.org/movie/747188-asteroid-city/watch?locale=US'])
(['A premonition helps save lives and Death seeks revenge in this horror-mystery starring Bobby Campo and Shantel VanSanten.', 'space shuttle, mass murder, cable car, space marine, ', 'Tough cop takes on vigilante justice-seekers in SF.', "If you like I Want to Eat Your Pancreas, you'll like this movie"], ['https://www.themoviedb.org/movie/19912-the-final-destination/watch?locale=US', 'https://www.themoviedb.org/movie/698-moonraker/watc

In [283]:
def get_movies(n):
    #get random page of results 
    page = random.randint(1, 500)
    url = "https://api.themoviedb.org/3/movie/popular?language=en-US&page="+str(page)
    response = requests.get(url, headers=headers)
    response_data = json.loads(response.text)
    random_movies = random.sample(response_data["results"], n)
    movie_text = defaultdict(str)

    for movie in random_movies:
        print(movie['title'])
        movie_text[str(movie['id'])] = movie['overview']

    for key in movie_text.keys():
        url = "https://api.themoviedb.org/3/movie/" + key + "/reviews?language=en-US&page=1"
        reviews = requests.get(url, headers=headers)
        review_data = json.loads(reviews.text)

        if review_data['results']: 
            review_text = review_data['results'][0]['content']
            movie_text[key] += review_text
    
    return movie_text

In [148]:
#get random page of results 
page = random.randint(1, 500)

In [280]:
url = "https://api.themoviedb.org/3/movie/popular?language=en-US&page="+str(page)

response = requests.get(url, headers=headers)
response_data = json.loads(response.text)

In [151]:
random_movies = random.sample(response_data["results"], 3)

In [152]:
movie_text = defaultdict(str)

for movie in random_movies:
    movie_text[str(movie['id'])] = movie['overview']
    
for key in movie_text.keys():
    url = "https://api.themoviedb.org/3/movie/" + key + "/reviews?language=en-US&page=1"
    reviews = requests.get(url, headers=headers)
    review_data = json.loads(reviews.text)
    
    if review_data['results']: 
        review_text = review_data['results'][0]['content']
        movie_text[key] += review_text
    

# Step 2: Ask GPT

In [273]:
def generate_description(movie_id, text, choice = 1):
    
    #generic genre based description + reviews
    if choice == 1:
        prompt = f"generate a 'blind date with a book' description for this movie:\
                \n\n{text}\n\n keep the description to maximum 10 words. Do NOT mention the movie's title."

        response = openai.Completion.create(
            engine="text-davinci-003",
            prompt=prompt,
            max_tokens=256
        )["choices"][0]["text"]
    
    
    #description based on movie recommendation 
    elif choice == 2:
        url = "https://api.themoviedb.org/3/movie/"+ movie_id + "/recommendations?language=en-US&page=1"
        recommend_response = requests.get(url, headers=headers)
        recommend_data = json.loads(recommend_response.text)
        if len(recommend_data['results']) > 0:
            recommend_movie = recommend_data['results'][0]['title']
            response = f"If you like {recommend_movie}, you'll like this movie"
        else:
            choice == 1
        
        
    #description based on movie keywords 
    elif choice == 3:
        url = "https://api.themoviedb.org/3/movie/"+ movie_id + "/keywords"
        keywords_response = requests.get(url, headers = headers)
        keywords_data = json.loads(keywords_response.text)
        keywords_response = random.sample(keywords_data['keywords'], min(4, len(keywords_data['keywords'])))
        keywords_list = []
        response = []
        
        for item in keywords_response:
            response.append(item['name'])
            
        if len(response) == 0:
            choice = 1
            
    
    #description based on actor
    elif choice == 4:
        
        url = "https://api.themoviedb.org/3/movie/"+ movie_id + "/credits?language=en-US"
        
        credits_response = requests.get(url, headers = headers)
        credits_data = json.loads(credits_response.text)
        actors = credits_data['cast'][0]['name'] + " and " + credits_data['cast'][1]['name'] 
        prompt = f"generate a very short description for this movie:\
                \n\n{text}\n\n Do NOT mention the movie's title. Make sure to mention BOTH {actors}.\
                Keep the description to maximum 10 words. "
        
        response = openai.Completion.create(
            engine = "text-davinci-003",
            prompt = prompt,
            max_tokens=256
        )["choices"][0]["text"]
        
    return response

In [274]:
blind_descriptions = []
for i, (key, val) in enumerate(movie_text.items()):
    blind_descriptions.append(generate_description(key, val, choice = i+1))
    
blind_descriptions

['\n\nTeen crush threatens world domination.',
 "If you like Superman/Batman: Apocalypse, you'll like this movie",
 ['london, england', 'based on novel or book', 'dover', 'orphan']]

In [278]:
def get_four_random():
    movie_text = get_movies(4)
    
    blind_descriptions = []
    for i, (key, val) in enumerate(movie_text.items()):
        blind_descriptions.append(generate_description(key, val, choice = i+1))
        
    return blind_descriptions

In [285]:
for i in range(5):
    print(i, get_four_random())

Undryable Younger Cousin
Dragnet
My Babysitter's a Vampire
The Golden Cockerel
0 ['\n\n"Sibling love, secrets, and betrayal."', "If you like The Man with One Red Shoe, you'll like this movie", ['vampire', 'babysitter', 'fan culture'], '\n\nLucha Villa and Ignacio Tarso battle for love.']
Leatherface: The Texas Chainsaw Massacre III
Dawn of the Dead
Training Day
Virgin Territory
1 ['\n\nRoad trip horror: Blood and mayhem!', "If you like Day of the Dead, you'll like this movie", ['california', 'drug trafficking', 'los angeles, california', 'rookie cop'], '\n\nRomantic swashbuckling with Hayden Christensen and Mischa Barton.']
Into the Wild
Unhappily Ever After
Warm Bodies
American Ultra
2 ['\n\nWild adventures in Alaskan wilderness.', "If you like Attack on Titan, you'll like this movie", ['dystopia', 'based on young adult novel', 'based on novel or book', 'interspecies romance'], '\n\nJesse and Kristen star in an action-packed comedy disaster.']
Invictus
All About Anna
The Courier
Ameri

# Letterboxd API